# Capstone Project 1: House Price Prediction (Regression)

---

## Learning Objectives

By completing this project you will be able to:

- Frame a real-world business problem as a supervised regression task
- Generate and explore a realistic synthetic dataset
- Build preprocessing pipelines with scikit-learn
- Train, evaluate, and compare multiple regression models
- Perform residual analysis and error diagnostics
- Tune hyperparameters with GridSearchCV
- Save and load trained models with joblib

## Prerequisites

- Python 3.8+
- Libraries: numpy, pandas, matplotlib, seaborn, scikit-learn, joblib
- Familiarity with regression concepts (linear regression, regularization, tree-based models)

## Table of Contents

1. [Problem Statement & Business Context](#1)
2. [Data Generation](#2)
3. [Exploratory Data Analysis](#3)
4. [Data Splitting](#4)
5. [Baseline Model](#5)
6. [Preprocessing Pipeline](#6)
7. [Model Training](#7)
8. [Evaluation & Comparison](#8)
9. [Error Analysis](#9)
10. [Hyperparameter Tuning](#10)
11. [Final Test Set Evaluation](#11)
12. [Model Saving](#12)
13. [Conclusions and Next Steps](#13)

<a id="1"></a>
## 1. Problem Statement & Business Context

**Scenario:** A real estate agency wants to provide accurate price estimates for houses listed on its platform. Currently, agents rely on intuition and comparable sales, which leads to inconsistent pricing and lost deals.

**Goal:** Build a regression model that predicts house prices based on measurable features (square footage, number of bedrooms, age of the house, etc.). An accurate model will:

- Help sellers set competitive listing prices
- Help buyers identify good deals
- Reduce time-on-market for listed properties

**Success Metric:** We aim for a model with an R-squared of at least 0.85 on held-out test data and a Mean Absolute Error (MAE) that is small relative to average house price.

<a id="2"></a>
## 2. Data Generation

We create a synthetic dataset of 500 houses with 8 features and a target price. The price is generated from a realistic formula with added noise to simulate real-world variability.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

np.random.seed(42)
n_samples = 500

# --- Feature generation ---
sqft = np.random.normal(1800, 500, n_samples).clip(600, 5000).astype(int)
bedrooms = np.random.choice([1, 2, 3, 4, 5], n_samples, p=[0.05, 0.15, 0.40, 0.30, 0.10])
bathrooms = np.random.choice([1, 1.5, 2, 2.5, 3, 3.5], n_samples, p=[0.10, 0.15, 0.30, 0.20, 0.15, 0.10])
age = np.random.exponential(15, n_samples).clip(0, 80).astype(int)
garage = np.random.choice([0, 1, 2, 3], n_samples, p=[0.15, 0.35, 0.40, 0.10])
neighborhood_score = np.random.uniform(1, 10, n_samples).round(1)
distance_to_center = np.random.exponential(8, n_samples).clip(0.5, 40).round(1)
has_pool = np.random.choice([0, 1], n_samples, p=[0.70, 0.30])

# --- Price formula (realistic coefficients + noise) ---
price = (
    50_000
    + 120 * sqft
    + 15_000 * bedrooms
    + 12_000 * bathrooms
    - 1_500 * age
    + 10_000 * garage
    + 8_000 * neighborhood_score
    - 3_000 * distance_to_center
    + 25_000 * has_pool
    + np.random.normal(0, 30_000, n_samples)  # noise
)
price = price.clip(50_000)  # no negative prices

df = pd.DataFrame({
    "sqft": sqft,
    "bedrooms": bedrooms,
    "bathrooms": bathrooms,
    "age": age,
    "garage": garage,
    "neighborhood_score": neighborhood_score,
    "distance_to_center": distance_to_center,
    "has_pool": has_pool,
    "price": price.round(-2)
})

print(f"Dataset shape: {df.shape}")
df.head(10)

In [ ]:
df.describe().round(2)

In [ ]:
df.info()

<a id="3"></a>
## 3. Exploratory Data Analysis

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df["price"], bins=30, edgecolor="black", alpha=0.7, color="steelblue")
axes[0].set_title("Distribution of House Prices")
axes[0].set_xlabel("Price ($)")
axes[0].set_ylabel("Count")

axes[1].boxplot(df["price"], vert=True)
axes[1].set_title("Box Plot of House Prices")
axes[1].set_ylabel("Price ($)")

plt.tight_layout()
plt.show()

print(f"Mean price:   ${df['price'].mean():,.0f}")
print(f"Median price: ${df['price'].median():,.0f}")

In [ ]:
# Feature distributions
numeric_cols = ["sqft", "bedrooms", "bathrooms", "age", "garage",
                "neighborhood_score", "distance_to_center", "has_pool"]

fig, axes = plt.subplots(2, 4, figsize=(18, 8))
for ax, col in zip(axes.ravel(), numeric_cols):
    ax.hist(df[col], bins=25, edgecolor="black", alpha=0.7, color="teal")
    ax.set_title(col)
plt.suptitle("Feature Distributions", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
corr = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", center=0, square=True)
plt.title("Feature Correlation Matrix")
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots: top 4 features vs price
top_features = corr["price"].drop("price").abs().sort_values(ascending=False).head(4).index.tolist()

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for ax, feat in zip(axes, top_features):
    ax.scatter(df[feat], df["price"], alpha=0.4, s=15, color="steelblue")
    ax.set_xlabel(feat)
    ax.set_ylabel("price")
    ax.set_title(f"{feat} vs price (r={corr.loc[feat, 'price']:.2f})")
plt.suptitle("Top Correlated Features vs Price", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

<a id="4"></a>
## 4. Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["price"])
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set:     {X_test.shape[0]} samples")

<a id="5"></a>
## 5. Baseline Model

We establish a baseline using `DummyRegressor` (mean strategy). Any useful model must substantially beat this.

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_regression(model, X_tr, y_tr, X_te, y_te, name="Model"):
    """Fit model and return evaluation metrics on train and test sets."""
    model.fit(X_tr, y_tr)
    y_pred_train = model.predict(X_tr)
    y_pred_test = model.predict(X_te)

    results = {
        "Model": name,
        "Train MAE": mean_absolute_error(y_tr, y_pred_train),
        "Test MAE": mean_absolute_error(y_te, y_pred_test),
        "Train RMSE": np.sqrt(mean_squared_error(y_tr, y_pred_train)),
        "Test RMSE": np.sqrt(mean_squared_error(y_te, y_pred_test)),
        "Train R2": r2_score(y_tr, y_pred_train),
        "Test R2": r2_score(y_te, y_pred_test),
    }
    return results

baseline = DummyRegressor(strategy="mean")
baseline_results = evaluate_regression(baseline, X_train, y_train, X_test, y_test, "Baseline (Mean)")

print("Baseline Performance:")
for k, v in baseline_results.items():
    if k != "Model":
        print(f"  {k}: {v:,.2f}")

<a id="6"></a>
## 6. Preprocessing Pipeline

All features in this dataset are numeric, so we apply `StandardScaler` through a pipeline. This ensures consistent preprocessing during cross-validation and final evaluation.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define a helper to build a pipeline with any regressor
def make_pipeline(model):
    return Pipeline([
        ("scaler", StandardScaler()),
        ("model", model)
    ])

<a id="7"></a>
## 7. Model Training

We train four models:
1. **Linear Regression** - simple, interpretable baseline
2. **Ridge Regression** - L2 regularization to handle multicollinearity
3. **Lasso Regression** - L1 regularization for potential feature selection
4. **Random Forest** - non-linear ensemble method

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

models = {
    "Linear Regression": make_pipeline(LinearRegression()),
    "Ridge (alpha=1.0)": make_pipeline(Ridge(alpha=1.0, random_state=42)),
    "Lasso (alpha=1.0)": make_pipeline(Lasso(alpha=1.0, random_state=42)),
    "Random Forest": make_pipeline(RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
}

all_results = [baseline_results]

for name, pipe in models.items():
    result = evaluate_regression(pipe, X_train, y_train, X_test, y_test, name)
    all_results.append(result)
    print(f"{name:25s}  |  Test MAE: ${result['Test MAE']:>10,.0f}  |  Test R2: {result['Test R2']:.4f}")

<a id="8"></a>
## 8. Evaluation & Comparison

In [ ]:
results_df = pd.DataFrame(all_results).set_index("Model")
results_df = results_df.round(2)
results_df.sort_values("Test R2", ascending=False)

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ["Test MAE", "Test RMSE", "Test R2"]
colors = ["#e74c3c", "#3498db", "#2ecc71"]

for ax, metric, color in zip(axes, metrics, colors):
    results_df[metric].sort_values().plot(kind="barh", ax=ax, color=color, edgecolor="black")
    ax.set_title(metric, fontsize=13)
    ax.set_xlabel(metric)

plt.suptitle("Model Comparison", fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Identify the best model by Test R2
best_model_name = results_df["Test R2"].idxmax()
print(f"Best model: {best_model_name} (Test R2 = {results_df.loc[best_model_name, 'Test R2']:.4f})")

best_pipeline = models[best_model_name]

<a id="9"></a>
## 9. Error Analysis

We examine the residuals of the best model to check for systematic patterns.

In [ ]:
y_pred_test = best_pipeline.predict(X_test)
residuals = y_test - y_pred_test

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Residual plot
axes[0].scatter(y_pred_test, residuals, alpha=0.5, s=20, color="steelblue")
axes[0].axhline(y=0, color="red", linestyle="--")
axes[0].set_xlabel("Predicted Price")
axes[0].set_ylabel("Residual")
axes[0].set_title("Residuals vs Predicted")

# Residual distribution
axes[1].hist(residuals, bins=30, edgecolor="black", alpha=0.7, color="teal")
axes[1].set_xlabel("Residual")
axes[1].set_ylabel("Count")
axes[1].set_title("Residual Distribution")

# Actual vs Predicted
axes[2].scatter(y_test, y_pred_test, alpha=0.5, s=20, color="steelblue")
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
axes[2].plot([min_val, max_val], [min_val, max_val], "r--", label="Perfect prediction")
axes[2].set_xlabel("Actual Price")
axes[2].set_ylabel("Predicted Price")
axes[2].set_title("Actual vs Predicted")
axes[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Worst predictions
error_df = X_test.copy()
error_df["actual_price"] = y_test.values
error_df["predicted_price"] = y_pred_test
error_df["abs_error"] = np.abs(residuals.values)

print("Top 10 Worst Predictions:")
error_df.sort_values("abs_error", ascending=False).head(10)[
    ["sqft", "bedrooms", "age", "actual_price", "predicted_price", "abs_error"]
].round(0)

<a id="10"></a>
## 10. Hyperparameter Tuning

We use `GridSearchCV` to fine-tune the best model.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Tune Random Forest (most likely best model)
rf_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", RandomForestRegressor(random_state=42, n_jobs=-1))
])

param_grid = {
    "model__n_estimators": [100, 200, 300],
    "model__max_depth": [None, 10, 20, 30],
    "model__min_samples_split": [2, 5, 10],
}

grid_search = GridSearchCV(
    rf_pipeline,
    param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=0
)

grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV R2:      {grid_search.best_score_:.4f}")

In [ ]:
# CV results summary (top 10)
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results[["param_model__n_estimators", "param_model__max_depth",
            "param_model__min_samples_split", "mean_test_score", "rank_test_score"]] \
    .sort_values("rank_test_score").head(10)

<a id="11"></a>
## 11. Final Test Set Evaluation

In [ ]:
final_model = grid_search.best_estimator_
y_final_pred = final_model.predict(X_test)

final_mae = mean_absolute_error(y_test, y_final_pred)
final_rmse = np.sqrt(mean_squared_error(y_test, y_final_pred))
final_r2 = r2_score(y_test, y_final_pred)

print("=" * 50)
print("  FINAL MODEL - Test Set Performance")
print("=" * 50)
print(f"  MAE:  ${final_mae:>10,.0f}")
print(f"  RMSE: ${final_rmse:>10,.0f}")
print(f"  R2:   {final_r2:>10.4f}")
print("=" * 50)
print(f"  MAE as % of median price: {final_mae / y_test.median() * 100:.1f}%")

<a id="12"></a>
## 12. Model Saving

In [ ]:
import joblib
import os

os.makedirs("saved_models", exist_ok=True)
model_path = "saved_models/house_price_model.joblib"
joblib.dump(final_model, model_path)
print(f"Model saved to: {model_path}")

# Verify by loading
loaded_model = joblib.load(model_path)
test_pred = loaded_model.predict(X_test[:3])
print(f"\nSample predictions from loaded model: {test_pred.round(0)}")

<a id="13"></a>
## 13. Conclusions and Next Steps

### Key Findings

- **Square footage** is the strongest predictor of price, followed by number of bedrooms and neighborhood score.
- **Random Forest** outperformed linear models, suggesting non-linear relationships exist in the data.
- Regularization (Ridge/Lasso) provided marginal improvement over plain Linear Regression, indicating low multicollinearity.
- The tuned model achieves strong generalization with an R-squared well above our 0.85 target.

### Next Steps

1. **Feature Engineering:** Add interaction terms (e.g., sqft per bedroom), polynomial features, or binned neighborhood categories.
2. **Additional Models:** Try Gradient Boosting (XGBoost, LightGBM) which often outperform Random Forest.
3. **Real Data:** Replace synthetic data with actual MLS listings for production use.
4. **Deployment:** Wrap the model in a REST API (Flask/FastAPI) for real-time predictions.
5. **Monitoring:** Set up drift detection to retrain the model as market conditions change.